### 0. Start Selenium/Chrome Session

In [ ]:
from api.selenium import Selenium
from bs4 import BeautifulSoup
import datetime
import re
import shutil
import time
import pandas as pd

In [ ]:
# set vars
all_jobs = []
tmp_folder = 'browser_cache/jupyter_notebook'

In [ ]:
url_login = 'https://linkedin.com'
url_listings = 'https://www.linkedin.com/jobs/search/?geoId=90000070&keywords=data%20scientist&location=New%20York%20City%20Metropolitan%20Area'

In [ ]:
# open new session
try:
    shutil.rmtree(tmp_folder)
except:
    pass

browser = Selenium(tmp_folder, browser=True)

----

### 1. Get Job URLs (JS Render)

In [ ]:
browser.get_url(url_login) # complete manual login .. use a VPN and burner account!

In [ ]:
all_jobs = []

In [ ]:
for i in range(0,101,25):
    current_url = url_listings + '&start=' + str(i) # increment page
    
    browser.get_url(current_url) # load page
    time.sleep(5) # allow full page load
    
    r = browser.get_inner_html('//ul[contains(@class, "jobs-search-results__list")]')
    jsoup = BeautifulSoup(r, 'html.parser')
    jobs = jsoup.find_all('li',class_='jobs-search-results__list-item')
    all_jobs.extend(list(jobs)) # add to list all_jobs
    
    print(current_url)
    print(len(all_jobs))

In [ ]:
# clean up
all_jobs_urls = ['https://linkedin.com/jobs/view/' + re.search('(.*?)jobPosting:(.{10})(.*)',str(x)).group(2) for x in all_jobs]

# save
df = pd.DataFrame(all_jobs_urls)
df.to_csv('data/job_urls.csv')

---

### 2. Get Job Listing Page

In [ ]:
browser.get_url(url_login) # complete manual login .. use a VPN and burner account!

In [ ]:
df = pd.read_csv('data/job_urls.csv')

In [ ]:
df = df.drop(df.columns[0],axis=1)

In [ ]:
cols = ['job_title','company_name','region','applicant_count',
        'employment_salary_type_level','company_size_industry',
        'date_queried','date_posted','description']

df_jobs = pd.DataFrame(columns=cols)

In [ ]:
# text cleanup
def clean(x):
    x = re.sub('\n',' ',x)
    x = re.sub('\s{2}',' ',x)
    x = x.strip()
    return x

In [ ]:
for i in range(0,len(df)):
    row = []
    current_url = df.iloc[i,0]
    
    browser.get_url(current_url)
    time.sleep(5) # allow full page load
    
    j = browser.get_inner_html('//body')
    jsoup = BeautifulSoup(j, 'html.parser') # make soup from java-rendered page
    
    # build current row
    row.append(clean(jsoup.find('h1').text)) # job title
    row.append(clean(jsoup.find('span', class_='jobs-unified-top-card__subtitle-primary-grouping').contents[1].text)) # company_name
    row.append(clean(jsoup.find('span', class_='jobs-unified-top-card__subtitle-primary-grouping').contents[3].text)) # region
    
    if jsoup.find('span', class_='jobs-unified-top-card__applicant-count'):
        row.append(clean(jsoup.find('span', class_='jobs-unified-top-card__applicant-count').text)) # applicant_count
    else: 
        row.append(clean(jsoup.find('span', class_='jobs-unified-top-card__bullet').text)) # applicant_count

    row.append(clean(jsoup.find_all('div', class_='jobs-unified-top-card__job-insight')[0].text)) # employment_salary_type_level
    row.append(clean(jsoup.find_all('div', class_='jobs-unified-top-card__job-insight')[1].text)) # company_size_industry
    row.append(datetime.datetime.now().strftime('%Y-%m-%d %H:%M')) # date_queried
    
    if jsoup.find('span', class_='jobs-unified-top-card__posted-date'):
        row.append(clean(jsoup.find('span', class_='jobs-unified-top-card__posted-date').text)) # date_posted
    else: 
        row.append(None) # date_posted
                  
    row.append(clean(jsoup.find('div',class_='jobs-description-content__text').contents[7].text)) # description
    
    df_jobs.loc[len(df_jobs)] = row # append row to df_jobs
    df_jobs.to_csv('data/job_listings.csv') # quick save
    print(len(df_jobs))

----